In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

COLUMNAS_GRD = [
    "RUT",
    "SEXO",
    # "EDAD_CANT",
    "PREVI",
    "BENEF",
    "NOM_HOPI_P",
    "FECHA_INGRESO",
    "FECHA_EGRESO",
    "ANO_EGR",
    "DIAS_ESTAD",
    "DIAG1",
    "DIAG2",
    "DIAG3",
    "INTERV_Q",
    "COD_FONASA",
    "INTERV_Q_PPAL",
    "INTERV_Q_2",
    "INTERV_Q_3",
    "T_DIAG1",
    "T_DIAG2",
    "T_DIAG3",
    "PROCED",
    "PROCED_PPAL",
    "PROCED_2",
    "PROCED_3",
]

COLUMNA_GRD_INTERNO = [
    "rut",
    "sexo_(cod)",
    "prevision_(cod)",
    "hospital_de_procedencia_(cod)",
    "fecha_ingreso",
    "fecha_de_egreso",
    "ano_de_egreso",
    "estancias_totales",
    "diag_01_principal_(cod)",
    "diag_01_principal_(des)",
    "diag_02_secundario_(cod)",
    "diag_02_secundario_(des)",
    "diag_03_secundario_(cod)",
    "diag_03_secundario_(des)",
    "proced_01_principal_(cod)",
    "proced_01_principal_(des)",
]

In [3]:
# Lee GRD del INT
df_grd_sabanas = pd.read_csv("../data/processed/grd_sabanas.csv", usecols=COLUMNAS_GRD)
# Formatea fechas de GRD
df_grd_sabanas["FECHA_INGRESO"] = pd.to_datetime(df_grd_sabanas["FECHA_INGRESO"])
df_grd_sabanas["FECHA_EGRESO"] = pd.to_datetime(df_grd_sabanas["FECHA_EGRESO"])

# Lee GRD Interno del INT
df_grd_interno = pd.read_csv("../data/processed/grd_interno.csv") #, usecols=COLUMNA_GRD_INTERNO)
# Formatea fechas de GRD
df_grd_interno["fecha_ingreso"] = pd.to_datetime(df_grd_interno["fecha_ingreso"])
df_grd_interno["fecha_de_egreso"] = pd.to_datetime(df_grd_interno["fecha_de_egreso"])

# Lee la base de pabellon del INT
df_pabellon = pd.read_csv(
    "../data/external/datos_limpios_pabellon.csv", encoding="latin-1", sep=";"
)
# Formatea fechas de Pabellon
df_pabellon["fecha"] = pd.to_datetime(df_pabellon["fecha"])

# Formatea duraciones de operaciones
df_pabellon["duracion"] = pd.to_timedelta(df_pabellon["duracion"])

C:\Users\javie\AppData\Local\Temp\ipykernel_26552\2934221659.py:2: DtypeWarning: Columns (80,156) have mixed types. Specify dtype option on import or set low_memory=False.
  df_grd_sabanas = pd.read_csv("../data/processed/grd_sabanas.csv", usecols=COLUMNAS_GRD)
C:\Users\javie\AppData\Local\Temp\ipykernel_26552\2934221659.py:8: DtypeWarning: Columns (2,4,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df_grd_interno = pd.read_csv("../data/processed/grd_interno.csv") #, usecols=COLUMNA_GRD_INTERNO)


In [4]:
# Limpia la base de pabellon, sacando las suspendidas y sin duracion
df_pabellon_limpio = df_pabellon.copy()
df_pabellon_limpio = df_pabellon_limpio.query("suspendida == 'NO' and duracion.notna()")

In [5]:
# Une la base de GRD y Pabellon segun el RUT del Paciente
grd_y_pabellon = df_grd_sabanas.merge(
    df_pabellon_limpio, how="inner", left_on="RUT", right_on="ID_PACIENTE"
)

# Filtra la union de GRD y Pabellon para dejar las Int. Q que se hicieron dentro del egreso
grd_y_pabellon = grd_y_pabellon.query("fecha >= FECHA_INGRESO and fecha <= FECHA_EGRESO")

In [6]:
resumen_duraciones = (
    grd_y_pabellon.groupby(["ANO_EGR", "DIAG1"])["duracion"].describe().reset_index()
)

In [7]:
COLUMNAS_A_MOSTRAR = [
    # "rut",
    "fecha_ingreso",
    "fecha_de_egreso",
    "ano_de_egreso",
    "estancias_totales",
    "diag_01_principal_(cod)",
    "diag_01_principal_(des)",
    "proced_01_principal_(cod)",
    "proced_01_principal_(des)",
    "fecha",
    "primer_diagnostico",
    "nombre_de_la_operacion",
    "duracion",
]

# Une la base de GRD Interno y Pabellon segun el RUT del paciente
grd_interno_y_pabellon = df_grd_interno.merge(
    df_pabellon_limpio, how="inner", left_on="rut", right_on="ID_PACIENTE"
)

# Filtra la union de GRD y Pabellon para dejar las Int. Q que se hicieron dentro del egreso
grd_interno_y_pabellon = grd_interno_y_pabellon.query(
    "fecha >= fecha_ingreso and fecha <= fecha_de_egreso"
)

In [8]:
AGRUPACION_A_OBTENER = [
    "ano_de_egreso",
    "diag_01_principal_(cod)",
    "diag_01_principal_(des)",
    "proced_01_principal_(cod)",
    "proced_01_principal_(des)",
]
# Obtiene resumen de duraciones por anio, diagnostico y tipo de operacion
resumen_grd_interno_duraciones_por_operacion = (
    grd_interno_y_pabellon.groupby(AGRUPACION_A_OBTENER)["duracion"].describe().reset_index()
)

# Ordena los resultados por la cantidad de int. q. realizadas para cada diag
resumen_grd_interno_duraciones_por_operacion = (
    resumen_grd_interno_duraciones_por_operacion.sort_values(
        ["ano_de_egreso", "diag_01_principal_(cod)", "diag_01_principal_(des)", "count"],
        ascending=False,
    )
)

In [17]:
resumen_grd_interno_duraciones_por_operacion.to_excel(
    "../data/interim/duraciones_de_int_q_por_diagnostico.xlsx", index=False
)

## Diferencias de egresos entre ambas bases de GRD

In [10]:
# Obtiene cantidad de egresos para cada base de datos
cantidad_egresos_sabanas = df_grd_sabanas.groupby("ANO_EGR").size()
cantidad_egresos_grd_interno = df_grd_interno.groupby("ano_de_egreso").size()

resumen_egresos = pd.DataFrame(
    {
        "egresos_sabanas": cantidad_egresos_sabanas,
        "egresos_grd_interno": cantidad_egresos_grd_interno,
    }
)

display(resumen_egresos)

,egresos_sabanas,egresos_grd_interno
2015,4621,4621
2016,4872,4872
2017,4901,4901
2018,5482,5482
2019,5435,5435
2020,2848,2848
2021,3086,3086
2022,3219,3219


Por lo tanto, ambas bases poseen la misma cantidad de egresos por año.

In [11]:
cantidad_de_int_q = df_pabellon.shape[0]
sin_suspendidas = df_pabellon.query("suspendida.isna()")
suspendidas = df_pabellon.query("suspendida == 'SI'")
suspendidas_pero_con_duracion = suspendidas.query("duracion.notna()")
no_suspendidas_pero_sin_duracion = df_pabellon.query("suspendida == 'NO' and duracion.isna()")

print(f"> Hay {cantidad_de_int_q} intervenciones quirurgicas en la BD.")
print(f"> De esas, {suspendidas.shape[0]} son suspendidas.")
print(f"> De las {suspendidas.shape[0]} suspendidas, hay "
      f"{suspendidas_pero_con_duracion.shape[0]} que tienen duracion de la operacion. "
      f"Estas seran eliminadas, pero hay que discutirlo.")
print(f"> Ademas, hay {no_suspendidas_pero_sin_duracion.shape[0]} intervenciones NO suspendidas, "
      f"pero sin tiempo. Estas seran eliminadas.")
print(f"> Finalmente, hay {sin_suspendidas.shape[0]} intervenciones que NO indican si fueron "
      f" suspendidas o no. Estas seran eliminadas, pero hay que discutirlo")
print(f"\nPor lo tanto, hay {df_pabellon_limpio.shape[0]} intervenciones validas.")

> Hay 15897 intervenciones quirurgicas en la BD.
> De esas, 1355 son suspendidas.
> De las 1355 suspendidas, hay 25 que tienen duracion de la operacion. Estas seran eliminadas, pero hay que discutirlo.
> Ademas, hay 10 intervenciones NO suspendidas, pero sin tiempo. Estas seran eliminadas.
> Finalmente, hay 26 intervenciones que NO indican si fueron  suspendidas o no. Estas seran eliminadas, pero hay que discutirlo

Por lo tanto, hay 14506 intervenciones validas.
